In [4]:
import spacy
import dhlab as dh

import citation_finder as cf
import re
from typing import List, Tuple, Dict, Union, Optional, Iterable, Callable, Any

from spacy import displacy
from spacy.tokens import Doc, Span
from spacy.training import Example
from spacy.matcher import Matcher
from spacy.language import Language

import pandas as pd


# Tren en Spacy spancat-modell for kildehenvisninger

1. Hent data: setninger med kildehenvisninger 
2. Test spancat vs. regex på data
3. Annoter data i prodigy
4. Splitt data i train, dev og test
5. Tren span_finder / suggester 
6. Tren spancat med den allerede trente span_finder

## 1) Forbered eksempeldata
Setningene her hentet fra følgende (tilfeldig valgte) tidsskrifter: 

- https://ntnuopen.ntnu.no/ntnu-xmlui/bitstream/handle/11250/2642231/Hjorth.pdf?sequence=1&isAllowed=y

- Anderssen og Lohndal, 2022, *Innledning til Norsk Lingvistisk Tidsskrifts
temahefte om grammatisk kjønn*, https://munin.uit.no/bitstream/handle/10037/25148/article.pdf?sequence=2 (visited 2023-07-26)

- https://munin.uit.no/bitstream/handle/10037/25139/article.pdf?sequence=2&isAllowed=y
 
- Urek et al., 2022, *"En splyv eller et splyv? Tilordning av
grammatisk genus til pseudosubstantiv
i norsk"*, *Norsk lingvistisk tidsskrift (trykt utg.). 2022 Vol. 40 Nr. 1*, https://ntnuopen.ntnu.no/ntnu-xmlui/handle/11250/3035622 (visited 2023-07-26)


- Grøtta et al., 2022, *Norsk litteraturvitenskapelig tidsskrift: Volume 25 | Issue 2*, https://www.idunn.no/doi/epdf/10.18261/nlvt.25.2 (visited 2023-07-26), DOI: http://doi.org/10.18261/issn.1504-288X

- Opsahl, 2019, *Aristokratiske allianser og konflikter i nordisk
seinmiddelalder*, https://ntnuopen.ntnu.no/ntnu-xmlui/bitstream/handle/11250/2651469/Opsahl.pdf?sequence=4&isAllowed=y (visited 2023-07-27)

In [32]:
texts = """
Videre omtaler han denne minneaktiviteten som «mediated action», karakterisert av at aktører handler og ytrer seg om fortiden, og danner minnefellesskap, ved hjelp av kulturelle redskaper eller artefakter (Wertsch, 2002, s. 13, 17, 62–65).
Det teoretiske rammeverket for analysen er en sosiokulturelt innrettet minneteori, slik kulturantropologen James V. Wertsch legger den fram i boka Voices of Collective Remembering (2002).
Min undersøkelse er et bidrag til denne typen minneforskning, som enkelte har foreslått å definere som et eget felt under merkelappen «media memory» (Neiger, Meyers & Zandberg, 2011).
...men at ulike typer medier bidrar til å forme minner på ulike måter (Wertsch, 2002, s. 52).
Sommeren 2017 besluttet regjeringen å avbryte arbeidet med å virkeliggjøre forslaget (KUD, 2017).
Videre fastslår den at bildene som illustrasjoner av dette tenkte minnestedet viser til et konkret, geografisk område (identifisert som et stykke Norge, nærmere bestemt landskapet omkring Utøya og Sørbråten), og til en spesifikk historisk hendelse (terroren 22. juli 2011).
Begge tilhører det Barthes (1994, s. 25) omtaler som «estetiske signifikater».
En rekke kompositoriske grep gir Dahlbergs illustrasjon en karakter som minner om 1800-tallets nordiske landskapsmaleri, slik det for eksempel er representert i Hans Gudes Vestlandsfjord fra 1862 (Ill. 4).
Bildene tillegger på den ene siden stedet bestemte egenskaper (dramatikk, storslagenhet) som det ikke nødvendigvis ville komme til å ha i virkeligheten (jf. den tidligere nevnte uttalelsen fra KORO om konkurransebildenes idealisering av motivet).
Også i vitenskapelige publikasjoner er Memory Wound sammenstilt med andre minnesteder og omtalt på måter som gjør plasseringen i tid og rom uklar (se f.eks. Meyer, 2015; De Turk, 2017; Heath-Kelly, 2017; Knudsen & Ifversen, 2017).
Corbett (1990: 1) åpner sin innflytelsesrike bok om grammatisk kjønn eller genus med å slå fast følgende:
Det kan ha med prosessering å gjøre, som Corbett (1990) allerede var inne på, altså at genus gjør det enklere å prosessere kommende informasjon i en setning, men gitt at så mange språk greier seg uten, er dette neppe en uttømmende forklaring på opphavet til genus.
Hocketts (1958: 231) klassiske definisjon viser dette tydelig.
Spørsmålet om hvorvidt det finnes noen regler for tilordning av genus i norsk er et tema som har vært mye diskutert opp gjennom årene (se for eksempel Trosterud 2001), og bidraget fra Urek, Lohndal og Westergaard (2022) tar opp nettopp dette problemet.
Dette skjer via en mekanisme som blir kalla sonde - mål-kongruens (sjå t.d. Chomsky 2000: 122).
Basert på korpusdata ser Rodina & Westergaard (2013) på korleis unge barn i Tromsø lærer seg kjønnssystemet.
Mange forskarar har etter kvart komme fram til dette: Fretheim (1985), Lødrup (2016, 2021), Svenonius (2017), Westergaard & Rodina (2016), Busterud et al. (2019, 2020).
Enger (2004) argumenterer for ein mellomposisjon der den bundne artikkelen til ein viss grad uttrykkjer genus (sjå også Berg 2019).
Et slikt perspektiv sier derfor svært lite om den psykologiske statusen til slike regler (jf. Gagliardi 2012: 110).
For å få innsikt i dette er det nødvendig å «uncover psychologically real and productive criteria that speakers exploit in “on-the-spot” gender assignment» (Thornton 2009: 17, jf. Corbett 1991 og Audring 2016).
Post hoc parvise sammenligninger (Lenth 2016) viste at det var signifikant større sannsynlighet for at deltakerne produserte hankjønn for pseudosubstantiver som endte på konsonant...
I Frankrike har man oppdaget at kristendommen også har en viktig plass i MichelHouellebecqs romanunivers (Julliot 2022).
I 1919 skrev Christian Claussen en artikkel i Edda om «Digteromvendelserne omkring Aarhun-dredskiftet» og trakk frem Garborg, August Strindberg og Johannes Jørgensen som skandi-naviske eksempler på diktere som hadde omvendt seg til katolisismen.
Huysmans’  roman  À  rebours  (1884),  oversatt  tilnorsk  under  tittelen  Mot  strømmen  (1998),  ble  straks  assosiert  med  dekadansen  og  harsenere til og med blitt kalt «den dekadente ånds Bibel» (Huysmans 1977, baksideteksten).
Men i Garborgs vokabular er «ny-idealistisk» og «dekadent» og «ny-idealister» og «dekadenter» tilnærmet synonymer (Garborg 1980 b, 423).
De navngitte utstederne var ridderen Sten Sture d.e. (ca. 1437‒1503), tidligere svensk riksforstander og på dette tidspunktet rikshovmester, og ridderen Knut Alvsson (ca. 1455‒1502), som titulerte seg «ridder i Norge», dessuten var «kopperbergsmenn», «sølvbergsmenn», «jernbergsmenn» og «menige allmue i Dalarne» ført opp som utstedere.
Kong Hans ble dessuten beskyldt for å ha fraternisert med russerne som hadde angrepet rikest østligste provins («landsende») Finland.
Det norske aristokratiet var derimot for svakt til å stå imot den danske dominansen innenfor unionen over tid, og Norge gikk til slutt under som et selvstendig rike (Moseng m.fl. 2007: 318‒400).
Dansk historiografi på si side har særlig lagt vekt på Sten Sture som den dyktige taktikeren (Schück 2013).
I løpet av 1900-tallet kom imidlertid tendenser til et endret syn, som kulminerte med Sven Ulric Palmes (1968) totale revurdering i sin biografi fra 1950.
Bo Jonsson ble Sveriges rikeste og mektigste mann i 1370- og 80-åra og var en viktig premissleverandør for at Sverige gikk inn i Kalmarunionen (Engström 1935; Larsson 1997: 36‒38).
Mistankens hermeneutikk er utvilsomt berettiget (Hellesnes 1988: 170).
 """.strip().split("\n")


In [33]:
# Lagre setningene til en txt-fil
with open("assets/citation_examples.txt", "w") as outfile:
    for sentence in texts:
        outfile.write(sentence + "\n")

## 2) Test vanlige regex vs. spacy span_ruler vs. default spancat på data

Sjekk hvilke tekstspenn vi får hentet med noen enkle regulære uttrykk, og sammenlign med spancat som  henter alle ngram (default `suggester`-funksjon, `n=[1,2,3]`)

In [22]:
# Last ned en norsk spacy-modell
 # sm kan byttes med md eller lg
#!spacy download nb_core_news_sm

In [40]:
# Last inn spacy
nlp=spacy.load("nb_core_news_sm")

In [41]:
# Configure colours to visualize with
def colour_options():
    return dict(
        colors={
            "REGEX":"#5f9100", # label for regex-matched spans
            "SPAN_RULER":"#ff5100", # label for matched spans with spacy rule patterns
            "SPANCAT":"#ff9120" # label for spancat-matched spans
        })


### REGEX

In [42]:
def regex_patterns():

    patterns = [
        # Ett eller flere årstall, valgfritt med sidetall: https://regex101.com/r/vr4Adl/1
        r"\(\d{4},? ?s?\.? ?\d*-?\d*\)",

        # Ett eller flere navn i parentes, med årstall: https://regex101.com/r/Od7g55/2
        r"\(([A-ZØÆÅ][a-zæøå]+( ?\d+[,: ]*))+\)",

        # Med eksplisitt henvisning (jf., se f.eks., sjå t.d.): https://regex101.com/r/z2CS6R/1
        r"\((jf\.|sjå også|se for eksempel|sjå t\.d\.) [A-ZØÆÅ][a-zæøå]+ \d+\:? ?\d+?\)",

        # Navn utenfor parentes, årstall inni: https://regex101.com/r/daYhiM/2
        r"([A-ZØÆÅ][a-zæøå]*,? [og& etal\.]*)+\((\d+[,:]? ?)+\)",
    ]

    return [re.compile(reg) for reg in patterns]

In [43]:
def match_regex_spans(text: str) -> list:
    """Apply all regex patterns on the texts, and only keep non-empty matches."""
    return [
        m.span()
        for regx in regex_patterns()
        for m in re.finditer(regx, text)
    ]

@Language.component("regex_span_finder")
def regex_span_finder(doc: Doc, spans_key: str = "regex") -> Doc:
    """Populate the doc.spans attribute with regex matches."""
    for (start, end) in match_regex_spans(doc.text):
        span= doc.char_span(start, end)
        if spans_key in doc.spans:
            doc.spans[spans_key] += (span,)
        else:
            doc.spans[spans_key] = (span,)
    return doc


In [44]:
nlp = spacy.load("nb_core_news_sm")
regex_ruler = nlp.add_pipe("regex_span_finder")
docs = list(nlp.pipe(texts))

for doc in docs:
    displacy.render(doc, style="span", options=colour_options(), jupyter=True)

/home/ingeridd/.pyenv/versions/3.12.0/envs/testing/lib/python3.12/site-packages/spacy/displacy/__init__.py:244: UserWarning: [W117] No spans to visualize found in Doc object with spans_key: 'sc'. If this is surprising to you, make sure the Doc was processed using a model that supports span categorization, and check the `doc.spans[spans_key]` property manually if necessary.

Available keys: []
  warnings.warn(Warnings.W117.format(spans_key=spans_key, keys=keys))


/home/ingeridd/.pyenv/versions/3.12.0/envs/testing/lib/python3.12/site-packages/spacy/displacy/__init__.py:244: UserWarning: [W117] No spans to visualize found in Doc object with spans_key: 'sc'. If this is surprising to you, make sure the Doc was processed using a model that supports span categorization, and check the `doc.spans[spans_key]` property manually if necessary.

Available keys: ['regex']
  warnings.warn(Warnings.W117.format(spans_key=spans_key, keys=keys))


### SPACY SPAN RULER

In [45]:
nlp = spacy.load("nb_core_news_sm")
ruler = nlp.add_pipe("span_ruler", config={"spans_key": "sc"})
label="SPAN_RULER"

patterns = [
    # Navn og ett eller flere årstall i parentes
    [{"TEXT": "("}, {"IS_ALPHA":True, "OP":"+"},{"IS_DIGIT":True, "LENGTH":4},{"IS_PUNCT":True, "OP": "?"},{"IS_DIGIT": True, "OP":"?"},{"TEXT": ")"}],
    # årstall i parentes
    [{"TEXT": "("}, {"IS_DIGIT":True, "LENGTH":4}, {"TEXT": ")"}],
    # (Potensielle) Navn utenfor parentes, årstall inni
    [{"ENT_TYPE":"PER", "IS_ALPHA":True, "OP":"*"},{"TEXT": "("},{"IS_DIGIT":True, "LENGTH":4}, {"TEXT": ")"}],
    # Substantiver/egennavn med stor forbokstav utenfor parentes, årstall inni
    [{"POS":{"IN": ["NOUN", "PROPN"]}, "IS_TITLE": True, "IS_ALPHA":True, "OP":"*", },{"TEXT": "("}, {"IS_ALPHA":True, "OP":"*"},{"IS_DIGIT":True, "LENGTH":4}, {"TEXT": ")"}],
    # Alt i parentes
    [{"TEXT": "("}, {"OP":"*"}, {"TEXT": ")"}],
]
spacy_patterns = [{"label": label, "pattern": pattern} for pattern in patterns]

ruler.add_patterns(spacy_patterns)


In [46]:
docs = list(nlp.pipe(texts))
for doc in docs[10:15]:
    displacy.render(doc, style="span", options=colour_options(), jupyter=True, )


In [47]:
# Save patterns to file with the "CITATION" label
ruler.remove(label)
ruler.add_patterns([{"label": "CITATION", "pattern": pattern} for pattern in patterns])
ruler.to_disk("assets/span_ruler")

### SPANCAT

In [48]:
nlp = spacy.load("nb_core_news_sm")

def create_examples(docs):
    for doc in docs:
        doc = add_char_spans(doc)
        yield Example(doc, doc)

examples = create_examples(docs) # Re-use the docs from the previous step

spancat = nlp.add_pipe("spancat")  # Load the defualt spancat component, which uses an ngram-suggester to suggest all possible uni-, bi- and trigrams
spancat.initialize(lambda: examples, nlp=nlp, labels=["SPANCAT"])


NameError: name 'add_char_spans' is not defined

In [ ]:
for doc in nlp.pipe(texts[10:15]):
    displacy.render(doc, style="span", options=colour_options(), jupyter=True, )


Ut fra de tre dummy-eksemplene med vanlige regex-mønstre, spacy span ruler, og default spancat, ser det ut som at vanlige regex gir oss de beste forslagene. 

## 3) Annoter data i Prodigy 

Sørg for at du har Prodigy installert først. **OBS!** Spør Ingerid eller Andre om lisensnøkler (`PRODIGY_LICENSE_KEY`) om du ikke allerede har en.

In [49]:
!pip install --upgrade prodigy -f https://${PRODIGY_LICENSE_KEY}@download.prodi.gy

Looking in links: https://****@download.prodi.gy
User for download.prodi.gy: ^C
ERROR: Operation cancelled by user


In [50]:
!pip install prodigy -f ~/local_packages/prodigy/1\.12\.0/


Looking in links: /home/ingeridd/local_packages/prodigy/1.12.0/
ERROR: Could not find a version that satisfies the requirement prodigy (from versions: none)
ERROR: No matching distribution found for prodigy


Kjør prodigy med [`spans.manual`](https://prodi.gy/docs/recipes#spans) og patterns som vi lagret tidligere.


In [28]:
!prodigy spans.manual citation_spans nb_core_news_lg ./assets/citation_examples.txt --label CITATION --patterns assets/span_ruler/patterns

Using 1 label(s): CITATION

✨  Starting the web server at http://0.0.0.0:8080 ...
Open the app in your browser and start annotating!

^C


## 4) Hent ut annotasjonene 

Lagre annotasjonene i `corpus/`og splitt i train, dev, test

In [37]:
!prodigy data-to-spacy --spancat citation_spans,eval:citation_spans -c configs/config.cfg corpus

ℹ Using language 'en'

============================== Generating data ==============================
Components: spancat
Merging training and evaluation data for 1 components
  - [spancat] Training: 36 | Evaluation: 36 (from datasets)
Training: 36 | Evaluation: 36
Labels: spancat (2)
✔ Saved 36 training examples
corpus/train.spacy
✔ Saved 36 evaluation examples
corpus/dev.spacy

============================= Generating config =============================
✔ Generated training config

======================== Generating cached label data ========================
✔ Saving label data for component 'spancat'
corpus/labels/spancat.json

============================= Finalizing export =============================
✔ Saved training config
corpus/config.cfg

To use this data for training with spaCy, you can run:
python -m spacy train corpus/config.cfg --paths.train corpus/train.spacy --paths.dev corpus/dev.spacy


## 6) Tren `spancat`-modell med annotasjonene


In [ ]:
# For rask eksperimentering med ulike konfigurasjoner (https://spacy.io/usage/training#config-stdin)
# | python -m spacy train --paths.train ./corpus/train.spacy --paths.dev ./corpus/dev.spacy

In [39]:
!python -m spacy init config - --lang nb --pipeline tok2vec,tagger,ner,span_ruler,spancat --optimize accuracy

[paths]
train = null
dev = null
vectors = "nb_core_news_lg"
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "nb"
pipeline = ["tok2vec","tagger","ner","span_ruler","spancat"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.span_ruler]
factory = "span_ruler"
annotate_ents = false
ents_filter = {"@misc":"spacy.first_longest_spans_filter.v1"}
matcher_fuzzy_compare 

In [38]:
!prodigy train training --spancat citation_spans --eval-split 0.2 -c configs/config.cfg --label-stats

ℹ Using CPU

========================= Generating Prodigy config =========================
✔ Generated training config

=========================== Initializing pipeline ===========================
[2023-07-28 14:53:55,351] [INFO] Set up nlp object from config
Components: spancat
Merging training and evaluation data for 1 components
  - [spancat] Training: 29 | Evaluation: 7 (20% split)
Training: 29 | Evaluation: 7
Labels: spancat (2)
[2023-07-28 14:53:55,384] [INFO] Pipeline: ['tok2vec', 'span_ruler', 'spancat']
[2023-07-28 14:53:55,391] [INFO] Created vocabulary
[2023-07-28 14:53:55,391] [INFO] Finished initializing nlp object
[2023-07-28 14:53:55,512] [INFO] Initialized pipeline components: ['tok2vec', 'span_ruler', 'spancat']
✔ Initialized pipeline

============================= Training pipeline =============================
Components: spancat
Merging training and evaluation data for 1 components
  - [spancat] Training: 29 | Evaluation: 7 (20% split)
Training: 29 | Evaluation: 7


# Citation finder

In [6]:
from citation_finder import findone, citation_finder

In [7]:
subject_word = "lingvistikk"
yearspan = (1900, 2020)

corpus = dh.Corpus(from_year=yearspan[0], to_year=yearspan[1], subject=subject_word)



In [8]:

citations = cf.citation_finder(corpus)

---

In [23]:
concs = cf.citation_finder(corpus, get_concs=True)

In [24]:
citations = cf.citation_finder(corpus)

In [34]:
citations = citations.rename(columns={0:"urn", 1:"citation"})

In [46]:
#citations["link"] = citations.apply(lambda c: f"https://urn.nb.no/{c['urn']}")
#citations.groupby(0).sum()
citations["link"] = citations["urn"].apply(lambda c: f"https://urn.nb.no/{c}")

In [51]:
citations.to_markdown()

'|     | urn                                            | citation                                                                   | link                                                             |\n|----:|:-----------------------------------------------|:---------------------------------------------------------------------------|:-----------------------------------------------------------------|\n|   0 | URN:NBN:no-nb_digibok_2012090408120            | Maegaard , Bente og Ruus , Hanne ( 1979 )                                  | https://urn.nb.no/URN:NBN:no-nb_digibok_2012090408120            |\n|   1 | URN:NBN:no-nb_digibok_2012090408120            | Bergen ( 1972 )                                                            | https://urn.nb.no/URN:NBN:no-nb_digibok_2012090408120            |\n|   2 | URN:NBN:no-nb_digibok_2012090408120            | Hellevik , Alf ( 1980 )                                                    | https://urn.nb.no/URN:NBN:no-nb_digibok_2012090408120    

In [11]:
tall = list(range(yearspan[0],yearspan[1]))

tallOR = ' OR '.join([str(x) for x in tall])

tallconc = dh.Concordance(corpus, tallOR, limit=4000)

concs = tallconc.frame
concs.concordance = concs.concordance.apply(lambda x:x.replace('<b>', '').replace('</b>', '').replace('...', ''))


In [50]:
!pip install tabulate

In [20]:
with open("concordances.md", "w") as fp:
    fp.write("\n \n- [ ] ".join(concs["link"].to_list()))

In [13]:

books = concs[['urn', 'concordance']]


    #### Regex1
#### finner alle konkordanser med parentes (eller semikolon) med årstall.

regex1 = r'[\(;].*?\D\d{4}\D.*?[\);]'

books['regex1'] = books1.concordance.apply(lambda x: findone(regex1, x))


NameError: name 'books1' is not defined

In [14]:
books

,urn,concordance
0,URN:NBN:no-nb_digitidsskrift_2018121081107_001,ling er sosialantropolog og har særlig in- An...
1,URN:NBN:no-nb_digibok_2012111508074,( Aksel Sandemose : Rejsen til Kjørkelvik ( 19...
2,URN:NBN:no-nb_digitidsskrift_2015012881007_001,"4 Ö Björnsson , EjöÖtrn 1908 , I , p. 44"
3,URN:NBN:no-nb_digibok_2012111508074,"Leve friheten ( 2010 ) , s . 44 ) ."
4,URN:NBN:no-nb_digibok_2012111508074,Da Per Aabel leste det i radio i 1993 ( han va...
...,...,...
941,URN:NBN:no-nb_digibok_2012111508074,Palme-morder skal visstnok foretrekkes ( Skri...
942,URN:NBN:no-nb_digitidsskrift_2018031681104_001,"Smetana sin etterfylgjar , Antonin Dvo ak , k..."
943,URN:NBN:no-nb_digitidsskrift_2018121081107_001,Fra 1930 - tallet til 1950 lå det tre fangelei...
944,URN:NBN:no-nb_digitidsskrift_2018031681104_001,Mål og Makt 1 - 2011


In [ ]:

books2 = books1[books1['parentes'] != 'itj no']
books2 = books2[['urn', 'concordance']]


In [22]:

#### Regex2
#### finner alle konkordanser som har tallkonstruksjoner med årstall og potensielt sidetall før sluttparentes (eller semikolon).
#### books3 er en dataramme med disse resultatene. Den er grunnlaget for å finne de to forskjellige typene siteringer med navn og
#### årstall inne i parentes, og med navn utenfor parentes.

regex2 = r'\[?\s*\d{4}\s*[a-zæøå]?\s*\.?\s*\]?(?:\s*[,-;:/\)\s]\s*(?:[ps]\s*\.?)?\[?\s*\d{1,4}\s*[a-zæøå]?\s*\.?\s*\]?)*\s*[\);]'

books['regex2'] = books.concordance.apply(lambda x: findone(regex2, x))


In [ ]:

books3 = books2[books2['årstall + sidetall'] != 'itj no']
books3 = books3[['urn', 'concordance']]


In [23]:

#### Navn i parentes
#### Regex3
#### finner alle konkordanser med navn, årstall og potensielt sidetall før sluttparentes (eller semikolon). Navnet kan følge "og/and/&"
#### eller følges av "et al.".

regex3 = r'(?:og|and|&)?(?:[A-ZÆØÅ](?:[A-ZÆØÅa-zæøå-]+|\s*\.)\s*,?\s*)+(?:\s*et\sal\.\s*)?,?\s*\[?\s*\d{4}\s*[a-zæøå]?\s*\.?\s*\]?(?:\s*[,-;:/\)\s]\s*(?:[ps]\s*\.?)?\[?\s*\d{1,4}\s*[a-zæøå]?\s*\.?\s*\]?)*\s*[\);]'

books['regex3'] = books.concordance.apply(lambda x: findone(regex3, x))


In [ ]:

if get_concs:
    return books3

books4 = books3[books3['navn i parentes'] != 'itj no']
books4 = books4[['urn', 'concordance']]


In [26]:

#### Regex4
#### finner alle treff i alle konkordanser som macther parenteser (eller semikolon) som består av minst én bokstav, årstall og hva som
#### helst annet. Antakelsen er at vi allerede har filtrert ut parentes-konstruksjoner som ligner på siteringer med navn, årstall og
#### sidetall og at alle parenteser med minimum én bokstav og et årstall derfor er siteringer.

regex4 = r'(?<=[\(;])[^(;\d]*?[A-ZÆØÅa-zæøå][^(;]*?\d{4}[^);]*?(?=[\);])'

match = []

for i in books.values:

    m = re.findall(regex4, i[1])
    if m != []:
        match.append((i[0],m))


match_df = pd.DataFrame(match)
match_explode = match_df.explode(column=1)


In [30]:

books["regex4"] = match_explode[1]


ValueError: cannot reindex on an axis with duplicate labels

In [31]:
books

,urn,concordance,regex1,regex2,regex3
0,URN:NBN:no-nb_digitidsskrift_2018121081081_001,"Birger Jåstad døydde seinhaustes 1984 , og du...",itj no,itj no,itj no
1,URN:NBN:no-nb_digitidsskrift_2018043081024_001,"( Finnur Jönsson 1884 , Sigurör Nordal , utg....","( Finnur Jönsson 1884 , Sigurör Nordal , utg. ...",1933 ),itj no
2,URN:NBN:no-nb_digitidsskrift_2011120155052_001,"Zeitschrift für deutsche Philologie , Bd 34 (...",( 1902 ),1902 ),itj no
3,URN:NBN:no-nb_digitidsskrift_2014112882038_001,", Holm-Olsen 1983 : 1 - 2. 28 Holm-Olsen 1970...",itj no,itj no,itj no
4,URN:NBN:no-nb_digitidsskrift_2014120182071_001,Søl ( Palmaria palmata ; fra L. Kristjånsson 1...,( Palmaria palmata ; fra L. Kristjånsson 1980 ),1980 ),L. Kristjånsson 1980 )
...,...,...,...,...,...
1327,URN:NBN:no-nb_digitidsskrift_2014120182071_001,"Holm-Olsen , Ludvig 1981 : Lys over norrøn ku...",itj no,itj no,itj no
1328,URN:NBN:no-nb_digitidsskrift_2011120155052_001,I sin uppsats Storken som livsbringare ( Forn...,( Fornvännen 1916 s . 104 ff . ),itj no,itj no
1329,URN:NBN:no-nb_digitidsskrift_2018043081024_001,) der til dels bygger på NBtKr ( Seip 1937 — ...,( Seip 1937 — 40 : 573 — 627 ;,itj no,itj no
1330,URN:NBN:no-nb_digitidsskrift_2018043081024_001,"dialekten i Oslo , som er en midt-østlandsk b...","( se Papazian og Helleland 2005 , pkt. 4.2.1.4 )",itj no,itj no


In [ ]:

#### Navn utenfor parentes
#### Regex5
#### tar utgangspunkt i books3 og finner alle konkordanser med navn utenfor parentes (eller semikolon) med årstall inni.

regex5 = '(?:[A-ZÆØÅ](?:[A-ZÆØÅa-zæøå-]+|\s*\.)\s*,?\s*)+[\(;]\s*\d{4}\D'

books3['navn u/parentes'] = books3.concordance.apply(lambda x: findone(regex5, x))

books5 = books3[books3['navn u/parentes'] != 'itj no']
books5 = books5[['urn', 'concordance']]

#### Regex6
#### finner alle treff i alle konkordanser som macther navn utenfor parentes (eller semikolon) som består av et årstall og hva som helst
#### annet. Navnekonstruksjonene kan bestå av flere navn etter hverandre, potensielt skilt av komma eller "og/and/&", eller etterfulgt
#### av "et al.".

regex6 = r'(?:(?:[A-ZÆØÅ](?:[A-ZÆØÅa-zæøå-]+|\s*\.)\s*,?\s*)+(?:og|and|&)\s*)?(?:[A-ZÆØÅ](?:[A-ZÆØÅa-zæøå-]+|\s*\.)\s*,?\s*)+(?:\s*et\sal\.)?\s*\(\s*\d{4}\D[^)]*?\)'

matchu = []

for i in books5.values:
    mu = re.findall(regex6, i[1])
    if mu != []:
        matchu.append((i[0],mu))

matchu_df = pd.DataFrame(matchu)
matchu_explode = matchu_df.explode(column=1)

### match_sorted
# setter sammen datarammene med siteringer for navn i og utenfor parentes, og sorterer på URN.

match_concat = pd.concat([match_explode, matchu_explode], ignore_index=True)
match_sorted = match_concat.sort_values(by=0, ignore_index=True)

return match_sorted









In [27]:
match_explode

,0,1
0,URN:NBN:no-nb_digitidsskrift_2018043081024_001,"Finnur Jönsson 1884 , Sigurör Nordal , utg. 1..."
1,URN:NBN:no-nb_digitidsskrift_2014120182071_001,fra L. Kristjånsson 1980
2,URN:NBN:no-nb_digitidsskrift_2018043081024_001,Seip 1955 : 129
3,URN:NBN:no-nb_digitidsskrift_2018043081024_001,Hine 1995 : 211 og Gladwell 2005 : 161
4,URN:NBN:no-nb_digitidsskrift_2014120182071_001,Heggstad 1977
...,...,...
269,URN:NBN:no-nb_digitidsskrift_2014120182071_001,"Dagsnytt i radio , 15 / 1 1979 , kl. 12,30 ."
270,URN:NBN:no-nb_digitidsskrift_2019052481110_001,"NKS-Forlaget , 1981"
271,URN:NBN:no-nb_digitidsskrift_2011120155052_001,Fornvännen 1916 s . 104 ff .
272,URN:NBN:no-nb_digitidsskrift_2018043081024_001,Seip 1937 — 40 : 573 — 627
